In [1]:
import os 
import pandas as pd
import glob

In [2]:
os.chdir('/Users/sivapalansarangan/Word2vec final/Data')

In [3]:
%pwd

'/Users/sivapalansarangan/Word2vec final/Data'

In [4]:
file_extension='.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

In [5]:
all_filenames[-1]

'மாமலர் - 24.csv'

In [6]:
len(all_filenames)

1941

In [6]:
df = pd.concat([pd.read_csv(f) for f in all_filenames])

In [7]:
os.chdir('/Users/sivapalansarangan/Word2vec final')

In [8]:
df.head()

,Heading,Chapter,Paragraph
0,சொல்வளர்காடு - 1,முதற்காடு : கௌஷீதகம்,தொன்மையான மலைநிலமாகிய கௌஷீதகத்தில் அசிதர் என்ன...
1,NaN,NaN,பாஞ்சாலநாட்டைச் சேர்ந்தவனும் கௌதம கோத்திரத்தவன...
2,NaN,NaN,மாதத்தில் ஒருநாள் தன் மாணவர்களின் தொழில்தேர்ச்...
3,NaN,NaN,அன்று அனல்முகப்பில் அவியிடல் முடிந்தபின் மிச்ச...
4,NaN,NaN,"விழிதிருப்பி அவனை நோக்கிய ஆசிரியர் “சொல், அது ..."


In [9]:
review_text=df["Paragraph"].values

In [10]:
import tensorflow as tf
import re
import string

In [11]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  #lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(input_data,
                                  '[%s]' % re.escape(string.punctuation+u'\u201c'+u'\u201d'+u'\u2018'+u'\u2019'), '')

In [12]:
def clean_text(text ): 
    delete_dict = {sp_character: ' ' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    
    return text2.split(' ')

In [13]:
newvec=[clean_text(a) for a in review_text]

In [103]:
#import nltk
#newvec=[nltk.word_tokenize(t) for t in review_text]

In [15]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
# init callback class
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        elif self.epoch % 100 == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        
        
        self.epoch += 1
        self.loss_previous_step = loss

/Users/sivapalansarangan/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [17]:
w2v_model = Word2Vec(vector_size = 300,
                     window = 10,
                     min_count = 2,
                     workers = 20,#
                     sg = 1,
                     negative = 5,
                     sample = 1e-5)

In [18]:
import time
start = time.time()
w2v_model.build_vocab(newvec)
w2v_model.train(newvec, 
                total_examples=w2v_model.corpus_count, 
                epochs=10, 
                report_delay=1,
                compute_loss = True, # set compute_loss = True
                callbacks=[callback()]) # add the callback class
end = time.time()

Loss after epoch 0: 2536859.5


In [15]:
from gensim.models import Word2Vec

/Users/sivapalansarangan/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [16]:
model=Word2Vec(newvec, min_count=2, vector_size=300,sg=1)# sg is o or 1 ie skkipgram or CRow

In [17]:
model.wv['முதற்புலரியில்']

array([-7.70000666e-02,  2.07129475e-02,  5.77804595e-02,  1.64391294e-01,
        9.07406732e-02, -2.00368091e-01,  4.82349843e-02,  2.04611778e-01,
        2.13105548e-02,  8.63581970e-02, -7.15428963e-02, -1.23663016e-01,
        1.21857800e-01,  2.10257247e-02, -4.08407785e-02, -3.96608263e-02,
        8.54550228e-02, -3.09147071e-02,  3.52958404e-02, -1.64251719e-02,
        5.51141165e-02,  5.27034365e-02,  5.21389060e-02,  8.90235230e-02,
        8.73655528e-02,  1.88452322e-02, -7.98982456e-02,  3.34160551e-02,
       -1.19487494e-01, -4.68719043e-02, -4.30517606e-02, -1.62199680e-02,
       -8.46163277e-03, -1.38674872e-02, -1.34320054e-02,  1.19105875e-01,
        3.55469100e-02, -1.44865185e-01,  2.40556933e-02, -2.27223225e-02,
       -1.66744545e-01,  4.77772690e-02, -6.15687631e-02, -2.86363438e-02,
       -1.80864073e-02,  1.68553129e-01,  4.76730801e-02, -4.76793423e-02,
        8.62214621e-03,  1.50339797e-01,  1.62687838e-01, -5.00383638e-02,
       -2.47858639e-04,  

In [18]:
model.wv.similarity('மாலையில்','காலை')

0.58557236

In [19]:
sims = model.wv.most_similar('காலை', topn=10)
sims

[('மாலை', 0.7511948943138123),
 ('புண்பட்ட', 0.6767891645431519),
 ('முதற்கதிர்', 0.6681026220321655),
 ('புலரியில்', 0.659150242805481),
 ('முற்புலரியில்', 0.651742160320282),
 ('அந்தியில்', 0.6479606628417969),
 ('மறுநாள்', 0.6350641250610352),
 ('பகல்', 0.6340986490249634),
 ('விடியற்காலையில்', 0.6293861269950867),
 ('“நாளை', 0.619310736656189)]

In [230]:
model.save("word2vect.model")

In [184]:
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [192]:
from gensim.models import KeyedVectors
#wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [232]:
model.wv.save_word2vec_format('word2vect.model')
import gensim
from gensim.scripts import word2vec2tensor
gensim.scripts.word2vec2tensor.word2vec2tensor('word2vect.model','word2vec_tensorfile', binary=False)